# 모델 매개변수 최적화

기본(pre-requisite) 코드

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

하이퍼파라미터(Hyperparameter)

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

최적화 단계(Optimization Loop)

손실 함수(loss function)

In [5]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

옵티마이저(Optimizer)

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.287954  [   64/60000]
loss: 2.284632  [ 6464/60000]
loss: 2.259821  [12864/60000]
loss: 2.265099  [19264/60000]
loss: 2.234807  [25664/60000]
loss: 2.208995  [32064/60000]
loss: 2.216933  [38464/60000]
loss: 2.180385  [44864/60000]
loss: 2.171770  [51264/60000]
loss: 2.143166  [57664/60000]
Test Error: 
 Accuracy: 43.8%, Avg loss: 2.134943 

Epoch 2
-------------------------------
loss: 2.139853  [   64/60000]
loss: 2.139470  [ 6464/60000]
loss: 2.069808  [12864/60000]
loss: 2.097278  [19264/60000]
loss: 2.033284  [25664/60000]
loss: 1.969401  [32064/60000]
loss: 2.008934  [38464/60000]
loss: 1.922656  [44864/60000]
loss: 1.925193  [51264/60000]
loss: 1.860207  [57664/60000]
Test Error: 
 Accuracy: 51.6%, Avg loss: 1.850896 

Epoch 3
-------------------------------
loss: 1.883140  [   64/60000]
loss: 1.860529  [ 6464/60000]
loss: 1.726487  [12864/60000]
loss: 1.784916  [19264/60000]
loss: 1.660083  [25664/60000]
loss: 1.616948  [32064/600